In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

## Sourcing the PDFs from URLs
I tried using many different methods to source the PDFs - all perfectly good in theory including:
* urllib2.request
* requests
* pdfminer - http://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python

...However the PDFs in the shark attack dataset are encoded content which gets corrupted if using any of the above.
Essentially I ended up with the following function which simply used curl to download the pdfs and to then decode the content.

In [3]:
import requests
import shutil

In [37]:
def get_the_pdf(url, loc):
    headers = {'User-Agent': 'curl'}
    response = requests.get(url, headers=headers, stream=True)

    if response.status_code == 200:
        with open(loc, 'wb') as output:
            response.raw.decode_content = True
            shutil.copyfileobj(response.raw, output)

## pdfminer function
The following function uses pdfminer to convert the pdf content into text format. The function takes just 1 argument being the pdf file path to convert.

In [15]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [33]:
# make an empty list to eventually hold all the attacks content from each pdf.
# This will later be the source of the DataFrame
attacks = []

In [34]:
# create tge DataFrame
sa = pd.read_csv('shark-attack-incurl.csv')

## Download the PDFs!
This may take ~15 mins...
Some faile so am performing this in a try / except block

In [ ]:
c = 0
with open(r'shark-attack-incurl.csv', 'r') as f:
    next(f)
    for l in f:
        c += 1
        if c >= 6086:
            break
        try:
            pdf = l.split(',')[-4]
            fn = l.split(',')[-6].replace(' ', '-')
                
            get_the_pdf(pdf, 'pdf/'+fn)
                
        except:
            print fn + ' failed, oh well move on'
            #p = pdf_from_url_to_txt(pdf)

"http://sharkattackfile.net/spreadsheets/pdf_directory/2014.10.17-Olsson failed, oh well move on
-pdf" failed, oh well move on
-pdf" failed, oh well move on
2014.01.04-Rianto.pdf failed, oh well move on
2004.04.05-JP-Andrew.pdf failed, oh well move on


In [153]:
sa['Case Number'] = sa['Case Number'].str.lower()

## Ingest into DF and prepare the PDFs
In this cell we loop through the Shark Attacks (sa) DataFrame and for each row we take the value in the pdf column (being the pdf file name) and pass it into the conert_pdf_to_txt function.
The through a series of nested list comprehension we prepared the data and add it to a list(l):
* filter the lines to only include those with a colon
* split the unstructured pdf text by newline
* split to produce a *key* and *value* pair (either side of the colon)
* make the key lowercase

Once prepared, we turn the list into a dictionary (d) and append each dictionary (representing the row) into a list.

In [54]:
import re

In [155]:
attacks = []
for r in sa.pdf:
    path = 'pdf/'+r
    try:
        p = convert_pdf_to_txt(path)
        l = [[i.strip().lower() for i in d.split(':')] for d in re.split('\\n', p) if ':' in d]
        d = {i[0]:i[1] for i in l}
        # add a key to the dictionary containing the pdf filename
        d['pdffilename'] = r
        d['Case Number'] = d['case'].replace('gsaf', '').strip() 
        attacks.append(d)
    except:
        pass

## Missing data
Sadly out of 6,085 shark attack, only 1,479 records had a pdf - this could be because some were corrupt or missing or had an invalid url.

In [119]:
len(attacks)

8

In [156]:
# Preview the data (list)
attacks[:5]

[{'': '',
  'Case Number': '2017.05.13',
  'activity': 'spearfishing / free diving',
  'case': 'gsaf 2017.05.13',
  'date': 'friday may 13, 2017',
  'depth': '18 metres, but the incident took place at the surface.',
  'description': 'a  41-year-old  male,  a  diabetic,',
  'distance from shore': 'five nautical miles',
  'distance from support boat': 'three metres',
  'environment': 'a video taken  on the  boat  shortly  before  the  incident  suggests the  two',
  'http': '//www.thenational.ae/uae/environment/20170514/emirati-fisherman-recovering-after-',
  'injury': 'right leg bitten',
  'location': 'the incident took place in the indian',
  'moon phase': 'full moon, 98% of the moon was illuminated.',
  'name': 'ali mohammad hamad al beloushi',
  'narrative': 'ali  mohammad  hamad  al  beloushi  and  his friend, omar,  left  khor  fakkan',
  'pdffilename': '2017.05.12-Beloushi.pdf',
  'sources': 'ahmed shaaban, khaleej times, may 13, 2017',
  'species': 'not  identified,  however,  it

In [157]:
# Load the list of pdf dictionaries into a DataFrame
attackspdf = DataFrame(attacks)

In [172]:
pdfdata = attackspdf[['Case Number', 'date','moon phase', 'time', 'weather', 'location', 'name', 'pdffilename']]

In [173]:
dd = sa.merge(pdfdata, how='inner', on='Case Number')

## The Merged Results! csv merged to pdf content in a DataFrame
The lower case column names are from the PDF doc - The Capital initialised column names are from the CSV

In [174]:
dd[:10][['Case Number', 'date','moon phase', 'time', 'weather', 'location', 'name', 'pdffilename', 'Type', 'Country', 'Location']]

,Case Number,date,moon phase,time,weather,location,name,pdffilename,Type,Country,Location
0,2017.05.06.r,"reported may 6, 2017",NaN,NaN,NaN,"near weipa, northern queensland, australia",josh 'ferrett' neille,2017.05.06.R-Neille.pdf,Provoked,AUSTRALIA,Weipa
1,2017.05.06,"saturday may 6, 2017",NaN,17h00,"at 16h52, cabo pulmo recorded 12-mile visibili...",the incident took place in the sea of cortez a...,andres rozada diego fernandez,2017.05.06-Rozada.pdf,Unprovoked,MEXICO,"Los Arbolitos, Cabo Pulmo"
2,2017.05.03,"wednesday may 3, 2017",first quarter,14h30,"at 14h53, knsa recorded scattered clouds and 10-",the incident took place in the pacific ...,sophia raab,2017.05.03-Raab.pdf,Invalid,USA,"Sunset Beach, Orange County"
3,2017.04.29.d,"saturday april 29, 2017","waxing crescent, 15% of the moon was illuminat...",17h24,NaN,the attack took place a surf spot calle...,leeanne ericson,2017.04.29.d-Ericson.pdf,Unprovoked,USA,"San Onofre, San Diego County"
4,2017.04.29.c,"saturday april 29,2017",NaN,17h10,"at 17h00, plettenberg bay recorded air tempera...",the incident took place in the indian ocean at...,,2017.04.29.c-Plett.pdf,Unprovoked,SOUTH AFRICA,"Keurbooms, Plettenberg Bay"
5,2017.04.29.a,"saturday april 29, 2017","new moon, april 26, 2017. last quarter, may",11h15,NaN,the attack took place in the indian,adrien dubosc,2017.04.29.a-Dubosc.pdf,Unprovoked,REUNION,Pointe au Sal
6,2017.04.27,"thursday april 27, 2017",NaN,14h00,NaN,"porpoise bay, catlins, south island,",,2017.04.27-NZ.pdf,Unprovoked,NEW ZEALAND,"Porpoise Bay, Catlins"
7,2017.04.22,"april 22, 2017","new moon, april 26, 2017",16h00,NaN,the incident took place in the south atlantic ...,frankie gonsalves,2017.04.22-Gonsalves.pdf,Unprovoked,"ST HELENA, British overseas territory",English Bay
8,2017.04.20,"thursday april 20, 2017",last quarter,08h50,"at 08h55, kgge georgetown county recorded an o...",the incident took place in the atlantic ...,,2017.04.20-PawleysIsland.pdf,Invalid,USA,Georgetown County
9,2017.04.17.b,"monday april 17, 2017",NaN,afternoon,NaN,the incident took place in the atlantic ocean ...,NaN,2017.04.17.b-Volusia.pdf,Unprovoked,USA,"Daytona Beach, Volusia County"


In [185]:
!pwd

/Users/lawrencefreeman/notebooks
